In [ ]:
api_key=""

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [3]:
from pinecone import Pinecone, ServerlessSpec
import os
index_name="hybrid-search-lanchain-pinecone"

pc = Pinecone(api_key="pcsk_GTbf4_HWzUpnHoTc5y1R535HgzEhJH3AWBXDw8WS7FRZ9nRNXdtU3XuahmbF2pobtVNN")

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
    name=index_name,
    dimension=384, 
    metric="dotproduct", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

    

/home/aryamantiwari17/GenAI/Langchain/Basic+advance RAG/myenv1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
index=pc.Index(index_name)
index

In [5]:
#vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-miniLM-L6-v2")
embeddings


HuggingFaceEmbeddings(model_name='all-miniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
embeddings

HuggingFaceEmbeddings(model_name='all-miniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
from pinecone_text.sparse import BM25Encoder
bm25=BM25Encoder().default()
bm25

In [8]:

sentences =[
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans",
]

## tfidf values on these sentence
bm25.fit(sentences)

## store the values to a json file
bm25.dump("bm25_values.json")

## load to your BM25Encoder object
bm_25_encoder = BM25Encoder().load("bm25_values.json")


100%|██████████| 3/3 [00:00<00:00, 48.29it/s]


In [9]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm_25_encoder,index=index)

In [10]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans",
    ]
)

100%|██████████| 1/1 [00:02<00:00,  2.51s/it]


In [11]:
retriever.invoke("what city i visted in 2022?")

[Document(metadata={'score': 0.39452064}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.305345714}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.285405248}, page_content='In 2021, I visited New Orleans')]